In [ ]:
#| default_exp nbdev_rt

In [ ]:
#| export
from __future__ import annotations

# nbdev_rt

> Real time [nbdev](https://nbdev.fast.ai/).


<!-- # Prologue -->

In [ ]:
#| export
import copy
import json
from typing import cast

import nbformat
from nbdev.process import extract_directives
from nbdev.process import NBProcessor


In [ ]:
#| export
from bridget.bridge import BridgePlugin
from bridget.bridge import get_bridge
from bridget.bridge_widget import bundled
from bridget.nb import IpynbConvertCB
from bridget.nb_state import get_nb
from bridget.nb_state import NB
from bridget.nb_state import this


In [ ]:
#|hide
from pathlib import Path

import fastcore.all as FC
from fastcore.test import *
from pote.common import shorten
from pote.display import RenderJSON
from rich.console import Console


In [ ]:
#|hide
from bridget.helpers import DetailsJSON
from bridget.helpers import in_vscode_notebook


In [ ]:
#| hide
cprint = (console := Console(width=120)).print

In [ ]:
nb = get_nb()
bridge = get_bridge()

In [ ]:
nb.cells

In [ ]:
nb.cells[10]

In [ ]:
#| export

class NBDev_rt(BridgePlugin):
    ctx_name = 'nbdev_rt'
    ctx_names = {'fetcher'}

    def on_state_update(self, *args, state:dict|str, **kwargs):
        # called after nbstate plugin has updated the state
        self.log(f"State update: {shorten(json.dumps(state), 'r', 100)}")

In [ ]:
bridge.logger.show(clear=True)

In [ ]:
bridge.add_plugins(nbrt := NBDev_rt())

In [ ]:
this(-3)

In [ ]:
nb = NB.from_NB(cast(NB, bridge.state))

nb.process(cb := IpynbConvertCB(in_vscode_notebook()))
ipynb_json = cb.notebook

nb = nbformat.reads(json.dumps(ipynb_json), as_version=nbformat.NO_CONVERT, capture_validation_error=(derr := {}))
test_eq(derr, {})

# DetailsJSON(ipynb_json)
RenderJSON(nb, init_level=1, max_height=400).display()

In [ ]:
nb.cells[14]

In [ ]:
procr = NBProcessor(nb=nb)
procr.process()

In [ ]:
from nbdev.maker import ModuleMaker
from nbdev.export import ExportModuleProc


In [ ]:
#| export

@FC.patch
def nb_export(self:NBDev_rt,
            nbname:str,        # Filename of notebook 
            nb:NB,
            lib_path:str=None, # Path to destination library.  If not in a nbdev project, defaults to current directory.
            procs=None,        # Processors to use
            name:str=None,     # Name of python script {name}.py to create.
            mod_maker=ModuleMaker,
            debug:bool=False,  # Debug mode
            solo_nb:bool=False # Export single notebook outside of an nbdev project.
            ):
    "Create module(s) from notebook state"
    if lib_path is None: lib_path = get_config().lib_path if is_nbdev() else '.'
    exp = ExportModuleProc()
    nb = NBProcessor(nbname, [exp]+L(procs), debug=debug)
    nb.process()
    for mod,cells in exp.modules.items():
        if FC.first(1 for o in cells if o.cell_type=='code'):
            all_cells = exp.in_all[mod]
            nm = FC.ifnone(name, getattr(exp, 'default_exp', None) if mod=='#' else mod)
            if not nm:
                warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
                     "Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.\n"
                     "See https://nbdev.fast.ai/getting_started.html for more information.")
                return
            mm = mod_maker(dest=lib_path, name=nm, nb_path=nbname, is_new=bool(name) or mod=='#', solo_nb=solo_nb)
            mm.make(cells, all_cells, lib_path=lib_path)

----
<!-- # Colophon -->

In [ ]:
#| hide
#| eval: false

import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean

In [ ]:
#| hide
#| eval: false

if FC.IN_NOTEBOOK:
    nb_path = '95_nbdev_rt.ipynb'
    nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)